In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers.models.bart import BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [3]:
basepath = "/data/jyk/aac_dataset/clotho/encodec/"
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')

In [4]:
basepath = "/data/jyk/aac_dataset/clotho/encodec/"
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')

In [5]:
data_files = {"train": "csv/train_short.csv", "validation": "csv/valid_short.csv"}

In [6]:
raw_dataset = load_dataset("csv", data_files=data_files)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-8533483370f473b7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 2/2 [00:00<00:00, 637.77it/s]


In [7]:
raw_dataset['train']

Dataset({
    features: ['file_path', 'caption'],
    num_rows: 19175
})

In [8]:
import numpy as np
import torch
import os

In [9]:
def preprocessing(example):
    path = example['file_path']
    encodec = np.load(os.path.join(basepath, path))
        
    target_text = tokenizer(text_target=example['caption'])
    labels = target_text['input_ids']

    #print('encodec',type(encodec))
    # print('labels', type(labels))
    model_input = {'input_ids': encodec , 'labels': labels}
    # print(type(model_input['input_ids']))
    return model_input


In [20]:
train_dataset = raw_dataset['train'].map(preprocessing, num_proc=16, )
train_dataset.set_format("pt", columns=['input_ids', 'labels'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8533483370f473b7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-e2f7646d8ef1d40a_*_of_00016.arrow


In [21]:
train_dataset[0]

{'input_ids': tensor([[206, 596, 996,  ..., 715, 571, 627],
         [ 16,  62, 178,  ..., 395, 723, 125],
         [ 16, 865, 297,  ..., 586, 568, 156],
         ...,
         [ 16,  54, 930,  ..., 529, 595, 877],
         [659, 858, 111,  ..., 375, 843, 364],
         [ 63, 204, 601,  ..., 918,  52, 329]]),
 'labels': tensor([    0,   250,   475, 23521,  6496,     9,  3187,  4238,     9,     5,
          1012,     2])}

In [22]:
valid_dataset = raw_dataset['validation'].map(preprocessing, num_proc=16)
valid_dataset.set_format('pt', columns=['input_ids', 'labels'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8533483370f473b7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-cb1bf448f3e38c57_*_of_00016.arrow


In [23]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="pt")

In [24]:
from torch.utils.data import DataLoader


In [25]:
print(train_dataset[0])
train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=data_collator, batch_size=8)

{'input_ids': tensor([[206, 596, 996,  ..., 715, 571, 627],
        [ 16,  62, 178,  ..., 395, 723, 125],
        [ 16, 865, 297,  ..., 586, 568, 156],
        ...,
        [ 16,  54, 930,  ..., 529, 595, 877],
        [659, 858, 111,  ..., 375, 843, 364],
        [ 63, 204, 601,  ..., 918,  52, 329]]), 'labels': tensor([    0,   250,   475, 23521,  6496,     9,  3187,  4238,     9,     5,
         1012,     2])}


In [28]:
for batch in train_dataloader:
    print(batch)

input_ids
[[[106, 928, 710, 360, 862, 19, 406, 1012], [257, 937, 653, 859, 680, 19, 491, 472], [855, 371, 653, 364, 537, 692, 772, 818], [257, 937, 618, 866, 680, 982, 866, 42], [904, 544, 819, 739, 685, 931, 628, 989], [855, 928, 937, 1018, 977, 634, 799, 835], [904, 363, 212, 493, 882, 763, 847, 237], [855, 841, 653, 889, 407, 634, 748, 237], [904, 741, 752, 859, 862, 430, 526, 975], [257, 404, 982, 830, 977, 239, 628, 42], [257, 363, 618, 780, 111, 185, 786, 534], [855, 302, 752, 215, 953, 887, 528, 989], [904, 937, 829, 889, 190, 494, 782, 343], [430, 928, 821, 215, 862, 634, 550, 558], [257, 859, 856, 956, 977, 19, 78, 393], [855, 424, 653, 558, 378, 461, 547, 322], [855, 841, 653, 919, 190, 634, 628, 472], [855, 974, 752, 866, 271, 982, 764, 989], [904, 133, 618, 1014, 683, 956, 114, 467], [855, 974, 1000, 255, 271, 634, 977, 701], [904, 974, 1000, 1022, 683, 887, 905, 237], [25, 144, 601, 815, 378, 461, 755, 617], [25, 870, 626, 1018, 701, 634, 905, 940], [25, 809, 842, 866, 38,

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).